In [1]:
from __future__ import division
import numpy as np
import scipy.linalg as slg
import math
import sys
from numpy import genfromtxt
import csv
from numpy import linalg as LA
from numpy.linalg import inv

In [2]:
def str2float(s):
    return float(s.replace('D','e'))

def CutLTMatOut(fname, head_line, index=0):
    cur_index = 0
    f = open(fname)
    status = None
    r = 0
    indices = []
    mat = []
    for line in f:
        if head_line in line:
            status = "Column Indices"
            continue
        if status == "Rows":
            splitted = line.split()
            if all([x.isnumeric() for x in splitted]):
                status = "Column Indices"
            else:
                try:
                    r = int(splitted[0])
                    floats = [str2float(s) for s in splitted[1:]]
                    if len(mat) < r:
                        mat.append(floats)
                    else:
                        mat[r-1] += floats
                except ValueError as e:
                    break
                continue
        if status == "Column Indices":
            status = "Rows"
            indices = [int(I) for I in line.split()]
            continue
    f.close()
    return mat

def toNPMatrix(mat_array, symmetry = 'symmetric'):
    dim = len(mat_array)
    mat = np.zeros((dim,dim))
    for i in range(dim):
        mat[i,:i+1] = mat_array[i]
    if symmetry == 'symmetric':
        diag = np.diag(np.diag(mat))
        return mat + mat.T - diag
    elif symmetry == 'anti-symmetric':
        return mat - mat.T


def ReadMatrix(fname, head_line, index=0):
    return np.array(CutLTMatOut(fname, head_line, index))

def ReadLowerTri(fname, head_line, index=0, symmetry = 'symmetric'):
    return toNPMatrix(CutLTMatOut(fname, head_line, index), symmetry)

def outMatrix(A):
    if np.iscomplexobj(A):
        print("Real part:")
        outMatrix(A.real)
        print()
        print("Imag part:")
        outMatrix(A.imag)
        return
    print("--------------------------------------------------------------------------------")
    curColStart, curColEnd = 0, 5
    while curColStart < A.shape[1]:
        curColEnd = min(curColStart + 5, A.shape[1])
        print("     "+"               ".join(list(map(str, range(curColStart + 1, curColEnd + 1)))))
        for i in range(A.shape[0]):
            print("%3d   "%(i+1,) + " ".join(list(map(lambda x : "%15.8e"%(x if abs(x) > 1e-8 else 0,) , A[i, curColStart : curColEnd]))))
        curColStart += 5
        print()

    print("--------------------------------------------------------------------------------")

def RMSD(V, W):
    """
    Calculate Root-mean-square deviation from two sets of vectors V and W.
    Parameters
    ----------
    V : array
        (N,D) matrix, where N is points and D is dimension.
    W : array
        (N,D) matrix, where N is points and D is dimension.
    Returns
    -------
    rmsd : float
        Root-mean-square deviation between the two vectors
    """
    diff = np.array(V) - np.array(W)
    N = len(V)
    return np.sqrt((diff * diff).sum() / N)
    

In [3]:
fname = "./neo_small_basis_stw.log"
elec_S_headline = 'Elec S Matrix:          0:'
elec_V_headline = 'Elec V Matrix'
prot_S_headline = 'Prot S Matrix:          0:'
prot_V_headline = 'Prot V Matrix'

# smat = ReadLowerTri(fname, head_line)

# print("Overlap Matrix: ")
# outMatrix(smat)

# # Diagonalize smat, and obtain the eigenvalues and eigenvectors
# Sigma_S, L_S  = LA.eig(smat)

# print("Eigenvalues of smat:")
# print(Sigma_S)

# print('Determinant of smat', np.linalg.det(smat))



# fname = "./hcn_ccpvdz_pb4.log"
# head_line = '*** NEO Overlap ***'

# smat_nd = ReadLowerTri(fname, head_line)

# print("Overlap Matrix (no D): ")
# outMatrix(smat_nd)

# # Diagonalize smat, and obtain the eigenvalues and eigenvectors
# Sigma_S_nd, L_S_nd  = LA.eig(smat_nd)

# print("Eigenvalues of smat (no D):")
# print(Sigma_S_nd)

# print('Determinant of smat (no D):', np.linalg.det(smat_nd))





#Sigma_S_minusOneHalf = (np.diag(Sigma_S**(-0.5)) )
#S_minusOneHalf = np.dot(L_S, np.dot(Sigma_S_minusOneHalf, np.transpose(L_S)))
#
#print("V: ")
#outMatrix(S_minusOneHalf)
#
#iden = np.dot(np.dot(S_minusOneHalf.transpose(), gs), S_minusOneHalf)
#
#print("Should be identity: ")
#outMatrix(iden)
#
#print("V+SV")
#
#iden2 = np.dot(np.dot(gv.transpose(), gs), gv)

In [4]:
elec_S = ReadLowerTri(fname, elec_S_headline)
elec_V = ReadMatrix(fname, elec_V_headline)
prot_S = ReadLowerTri(fname, prot_S_headline)
prot_V = ReadMatrix(fname, prot_V_headline)

In [5]:
print("Electronic Overlap Matrix: ")
outMatrix(elec_S)

Electronic Overlap Matrix: 
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    1.00000000e+00  2.48362000e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    2.48362000e-01  1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
  6    1.01528000e-06  3.61821000e-02  0.00000000e+00  0.00000000e+00  6.08130000e-02
  7    3.53823000e-02  3.59354000e-01  0.00000000e+00  0.00000000e+00  4.38893000e-01
  8    0.00000000e+00  0.00000000e+00  2.07283000e-01  0.00000000e+00  0.00000000e+00
  9    0.00000000e+00  0.00000000e+00  0.00000000e+00  2.07283000e-01  0.00000000e+00
 10   -5.94891000e-02 -3.19191000e-01  0.00000

In [6]:
print("Electronic Transformation Matrix: ")
outMatrix(elec_V)

Electronic Transformation Matrix: 
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    1.02865000e+00 -1.68595000e-01  0.00000000e+00  0.00000000e+00 -2.17409000e-03
  2   -1.68595000e-01  1.47126000e+00  0.00000000e+00  0.00000000e+00  3.09465000e-02
  3    0.00000000e+00  0.00000000e+00  1.22603000e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  1.01664000e+00  0.00000000e+00
  5   -2.17409000e-03  3.09465000e-02  0.00000000e+00  0.00000000e+00  1.25045000e+00
  6   -1.28103000e-03  2.54158000e-02  0.00000000e+00  0.00000000e+00  2.07374000e-02
  7    2.72029000e-02 -2.95573000e-01  0.00000000e+00  0.00000000e+00 -3.50423000e-01
  8    0.00000000e+00  0.00000000e+00 -1.23839000e-01  0.00000000e+00  0.00000000e+00
  9    0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.06522000e-01  0.00000000e+00
 10   -8.61304000e-03  2.47594000e-01  

In [7]:
Sigma_elec_S, L_elec_S  = LA.eig(elec_S)
elec_Sigma_minusOneHalf = (np.diag(Sigma_elec_S**(-0.5)) )
elec_V_manual = np.dot(L_elec_S, np.dot(elec_Sigma_minusOneHalf, np.transpose(L_elec_S)))

print("Manually calculated Electronic Transfromation Matrix V: ")
outMatrix(elec_V_manual)

Manually calculated Electronic Transfromation Matrix V: 
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    1.02865052e+00 -1.68594224e-01  0.00000000e+00  0.00000000e+00 -2.17404032e-03
  2   -1.68594224e-01  1.47125581e+00  0.00000000e+00  0.00000000e+00  3.09461452e-02
  3    0.00000000e+00  0.00000000e+00  1.22602992e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  1.01663572e+00  0.00000000e+00
  5   -2.17404032e-03  3.09461452e-02  0.00000000e+00  0.00000000e+00  1.25045269e+00
  6   -1.28102697e-03  2.54158838e-02  0.00000000e+00  0.00000000e+00  2.07374246e-02
  7    2.72027981e-02 -2.95572944e-01  0.00000000e+00  0.00000000e+00 -3.50423851e-01
  8    0.00000000e+00  0.00000000e+00 -1.23838757e-01  0.00000000e+00  0.00000000e+00
  9    0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.06522430e-01  0.00000000e+00
 10   -8.61294515

In [8]:
iden = np.dot(np.dot(elec_V_manual.transpose(), elec_S), elec_V_manual)

print("Electronic V+SV, Should be identity: ")
outMatrix(iden)

Electronic V+SV, Should be identity: 
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  9    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 10    0.00000000e+00  0.00000000e+0

In [9]:
elec_V_RMSD = RMSD(elec_V, elec_V_manual)
print(elec_V_RMSD)

3.381792704089167e-06


In [10]:
print("Protonic Overlap Matrix: ")
outMatrix(prot_S)

Protonic Overlap Matrix: 
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000000e+00  0

In [11]:
print("Protonic Transformation Matrix: ")
outMatrix(prot_V)

Protonic Transformation Matrix: 
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000000

In [12]:
Sigma_prot_S, L_prot_S  = LA.eig(prot_S)
prot_Sigma_minusOneHalf = (np.diag(Sigma_prot_S**(-0.5)) )
prot_V_manual = np.dot(L_prot_S, np.dot(prot_Sigma_minusOneHalf, np.transpose(L_prot_S)))

print("Manually calculated Protonic Transfromation Matrix V: ")
outMatrix(prot_V_manual)

Manually calculated Protonic Transfromation Matrix V: 
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00

In [13]:
iden = np.dot(np.dot(prot_V_manual.transpose(), prot_S), prot_V_manual)

print("Protonic V+SV, Should be identity: ")
outMatrix(iden)

Protonic V+SV, Should be identity: 
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000

In [14]:
prot_V_RMSD = RMSD(prot_V, prot_V_manual)
print(prot_V_RMSD)

0.0


In [15]:
Fe1_headline = 'Fe1 from l501 after NEO contribution:          0:'
Fe1 = ReadLowerTri(fname, Fe1_headline)
Pe1_headline = 'New Electronic  Density1          0:'
Pe1 = ReadLowerTri(fname, Pe1_headline)
e1_headline = 'Electronic Alpha error matrix at iteration1:'
e1 = ReadLowerTri(fname, e1_headline)

In [16]:
print("First Elec F:")
outMatrix(Fe1)

First Elec F:
--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -1.09546000e+01 -2.96062000e+00  0.00000000e+00  0.00000000e+00 -7.78200000e-03
  2   -2.96062000e+00 -1.60510000e+00  0.00000000e+00  0.00000000e+00 -1.13544000e-01
  3    0.00000000e+00  0.00000000e+00 -3.70740000e-01  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.53688000e-02  0.00000000e+00
  5   -7.78200000e-03 -1.13544000e-01  0.00000000e+00  0.00000000e+00 -5.36826000e-01
  6   -2.88185000e-04 -7.98664000e-01  0.00000000e+00  0.00000000e+00 -1.33751000e+00
  7   -4.35579000e-01 -8.68239000e-01  0.00000000e+00  0.00000000e+00 -9.68075000e-01
  8    0.00000000e+00  0.00000000e+00 -2.48253000e-01  0.00000000e+00  0.00000000e+00
  9    0.00000000e+00  0.00000000e+00  0.00000000e+00 -3.53775000e-01  0.00000000e+00
 10    7.25680000e-01  6.36941000e-01  0.00000000e+00  0.000

In [17]:
print("First Elec P:")
outMatrix(Pe1)

First Elec P:
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    1.03824000e+00 -1.28072000e-01  0.00000000e+00  0.00000000e+00  3.13777000e-02
  2   -1.28072000e-01  5.41473000e-01  0.00000000e+00  0.00000000e+00 -1.38950000e-01
  3    0.00000000e+00  0.00000000e+00  5.10508000e-01  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  3.11215000e-01  0.00000000e+00
  5    3.13777000e-02 -1.38950000e-01  0.00000000e+00  0.00000000e+00  3.19930000e-01
  6    6.64464000e-03 -1.47091000e-03  0.00000000e+00  0.00000000e+00 -2.80242000e-02
  7   -7.86003000e-03 -8.74930000e-02  0.00000000e+00  0.00000000e+00  1.40519000e-02
  8    0.00000000e+00  0.00000000e+00  3.18115000e-03  0.00000000e+00  0.00000000e+00
  9    0.00000000e+00  0.00000000e+00  0.00000000e+00  4.02954000e-01  0.00000000e+00
 10    7.15285000e-02 -1.65466000e-01  0.00000000e+00  0.000

In [18]:
FP_manual = np.dot(Fe1, Pe1)
outMatrix(FP_manual)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -1.08599071e+01 -4.81710778e-01  0.00000000e+00  0.00000000e+00 -2.01373373e-02
  2   -2.74888871e+00 -6.94334231e-01  0.00000000e+00  0.00000000e+00  4.59212045e-02
  3    0.00000000e+00  0.00000000e+00 -3.43705533e-01  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.50450202e-01  0.00000000e+00
  5   -2.02418334e-03  8.28120024e-02  0.00000000e+00  0.00000000e+00 -2.97477897e-01
  6   -1.95813381e-02  1.98091326e-01  0.00000000e+00  0.00000000e+00  1.86436864e-01
  7   -3.53775906e-01 -1.46722237e-01  0.00000000e+00  0.00000000e+00 -1.13229941e-01
  8    0.00000000e+00  0.00000000e+00 -1.48247677e-01  0.00000000e+00  0.00000000e+00
  9    0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.46474664e-01  0.00000000e+00
 10    6.38032773e-01  2.87372834e-01  0.00000000e+00  0.00000000e+00  1.5

In [19]:
outMatrix(elec_S)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    1.00000000e+00  2.48362000e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    2.48362000e-01  1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
  6    1.01528000e-06  3.61821000e-02  0.00000000e+00  0.00000000e+00  6.08130000e-02
  7    3.53823000e-02  3.59354000e-01  0.00000000e+00  0.00000000e+00  4.38893000e-01
  8    0.00000000e+00  0.00000000e+00  2.07283000e-01  0.00000000e+00  0.00000000e+00
  9    0.00000000e+00  0.00000000e+00  0.00000000e+00  2.07283000e-01  0.00000000e+00
 10   -5.94891000e-02 -3.19191000e-01  0.00000000e+00  0.00000000e+00 -3.1

In [20]:
FPS_manual = np.dot(np.dot(Fe1, Pe1), elec_S)
outMatrix(FPS_manual)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -1.09532842e+01 -2.97555846e+00  0.00000000e+00  0.00000000e+00 -1.45093795e-02
  2   -2.95530040e+00 -1.66937049e+00  0.00000000e+00  0.00000000e+00 -1.69052290e-01
  3    0.00000000e+00  0.00000000e+00 -4.68545072e-01  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.90828704e-01  0.00000000e+00
  5   -5.71909896e-03 -1.64150102e-01  0.00000000e+00  0.00000000e+00 -7.24765341e-01
  6   -8.44594229e-04 -7.90095369e-01  0.00000000e+00  0.00000000e+00 -1.31986808e+00
  7   -4.36804937e-01 -8.48995585e-01  0.00000000e+00  0.00000000e+00 -9.26557454e-01
  8    0.00000000e+00  0.00000000e+00 -2.18755502e-01  0.00000000e+00  0.00000000e+00
  9    0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.85786273e-01  0.00000000e+00
 10    7.30421175e-01  5.63883291e-01  0.00000000e+00  0.00000000e+00  2.6

In [21]:
FPS1_manual = np.dot(np.dot(Fe1, Pe1), elec_S)
e1_manual_AO = (FPS1_manual - FPS1_manual.transpose())
outMatrix(e1_manual_AO)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00 -2.02580540e-02  0.00000000e+00  0.00000000e+00 -8.79028051e-03
  2    2.02580540e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.90218706e-03
  3    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  5    8.79028051e-03  4.90218706e-03  0.00000000e+00  0.00000000e+00  0.00000000e+00
  6   -1.54314468e-04  1.07377915e-02  0.00000000e+00  0.00000000e+00  2.17372347e-02
  7   -3.72053255e-03  4.95591498e-03  0.00000000e+00  0.00000000e+00  1.09607321e-02
  8    0.00000000e+00  0.00000000e+00 -9.16048794e-03  0.00000000e+00  0.00000000e+00
  9    0.00000000e+00  0.00000000e+00  0.00000000e+00  4.01984013e-02  0.00000000e+00
 10    1.74457217e-02 -4.53858206e-03  0.00000000e+00  0.00000000e+00 -2.2

In [22]:
outMatrix(elec_V)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    1.02865000e+00 -1.68595000e-01  0.00000000e+00  0.00000000e+00 -2.17409000e-03
  2   -1.68595000e-01  1.47126000e+00  0.00000000e+00  0.00000000e+00  3.09465000e-02
  3    0.00000000e+00  0.00000000e+00  1.22603000e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  1.01664000e+00  0.00000000e+00
  5   -2.17409000e-03  3.09465000e-02  0.00000000e+00  0.00000000e+00  1.25045000e+00
  6   -1.28103000e-03  2.54158000e-02  0.00000000e+00  0.00000000e+00  2.07374000e-02
  7    2.72029000e-02 -2.95573000e-01  0.00000000e+00  0.00000000e+00 -3.50423000e-01
  8    0.00000000e+00  0.00000000e+00 -1.23839000e-01  0.00000000e+00  0.00000000e+00
  9    0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.06522000e-01  0.00000000e+00
 10   -8.61304000e-03  2.47594000e-01  0.00000000e+00  0.00000000e+00  2.5

In [23]:
# FPS-SPF
e1_manual_ON = np.dot(elec_V.transpose(), np.dot(e1_manual_AO, elec_V))
print("Error Matrix (in orthonormal basis):")
outMatrix(e1_manual_ON)

Error Matrix (in orthonormal basis):
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00 -5.27405898e-02  0.00000000e+00  0.00000000e+00 -8.50879737e-03
  2    5.27405898e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.48420862e-02
  3    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  5    8.50879737e-03  1.48420862e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
  6   -7.22556649e-04  2.15562046e-02  0.00000000e+00  0.00000000e+00  3.21292142e-02
  7   -1.70991490e-02  9.08095268e-03  0.00000000e+00  0.00000000e+00  1.38594890e-02
  8    0.00000000e+00  0.00000000e+00 -5.89846935e-03  0.00000000e+00  0.00000000e+00
  9    0.00000000e+00  0.00000000e+00  0.00000000e+00  4.10912059e-02  0.00000000e+00
 10    2.90418225e-02 -1.47264691e-02

In [24]:
e1_manual_ON_0p5 = e1_manual_ON *0.5
outMatrix(e1_manual_ON_0p5)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00 -2.63702949e-02  0.00000000e+00  0.00000000e+00 -4.25439868e-03
  2    2.63702949e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00 -7.42104308e-03
  3    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  5    4.25439868e-03  7.42104308e-03  0.00000000e+00  0.00000000e+00  0.00000000e+00
  6   -3.61278325e-04  1.07781023e-02  0.00000000e+00  0.00000000e+00  1.60646071e-02
  7   -8.54957451e-03  4.54047634e-03  0.00000000e+00  0.00000000e+00  6.92974452e-03
  8    0.00000000e+00  0.00000000e+00 -2.94923467e-03  0.00000000e+00  0.00000000e+00
  9    0.00000000e+00  0.00000000e+00  0.00000000e+00  2.05456030e-02  0.00000000e+00
 10    1.45209113e-02 -7.36323454e-03  0.00000000e+00  0.00000000e+00 -1.3

In [25]:
print(np.trace(np.dot(e1_manual_ON_0p5,e1_manual_ON_0p5)))

-0.013256310938513583


In [26]:
def getErrorMatrix(F,P,S,V,printlevel):
    FPS = np.dot(np.dot(F, P), S)

    e_AO = FPS - FPS.transpose()

    e_ON = np.dot(V.transpose(), np.dot(e_AO, V))

    if (printlevel >= 1):
        if (printlevel >= 2):
            print('F')
            outMatrix(F)
            print('P')
            outMatrix(P)
            print('FP')
            outMatrix(np.dot(F, P))
            print('S')   
            outMatrix(S)
        print('FPS')
        outMatrix(FPS)
        print('e_AO')
        outMatrix(e_AO)
        if (printlevel >= 2):
            print('V')
            outMatrix(V)
        print('e_ON')
        outMatrix(e_ON)
    return 0.5 * e_ON

In [27]:
def getBmatrix(E_list):
    Bsize = len(E_list)
    B = np.zeros((Bsize+1,Bsize+1))
    
    B[0,0] = 0.00
    for i in range(1,Bsize+1):
        B[i,0] = -1
        B[0,i] = -1
        B[i,i] = np.trace(np.dot(E_list[i-1],E_list[i-1]))
        for j in range(i+1,Bsize+1):
            B[i,j] = np.trace(np.dot(E_list[i-1],E_list[j-1]))
            B[j,i] = B[i,j]
        
    return B

In [33]:
E_list = [e1_manual_ON_0p5]
B = getBmatrix(E_list)
outMatrix(B)

--------------------------------------------------------------------------------
     1               2
  1    0.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00 -1.32563109e-02

--------------------------------------------------------------------------------


In [34]:
print('Determinant of B', np.linalg.det(B))

Determinant of B -1.0


In [35]:
Fe2_headline = 'Fe2 from l501 after NEO contribution:          0:'
Fe2 = ReadLowerTri(fname, Fe2_headline)
Pe2_headline = 'New Electronic  Density2          0:'
Pe2 = ReadLowerTri(fname, Pe2_headline)
e2_headline = 'Electronic Alpha error matrix at iteration for IMat=    2'
e2 = ReadLowerTri(fname, e2_headline)
e2_manual_ON_0p5 = getErrorMatrix(Fe2,Pe2,elec_S,elec_V,0)

In [36]:
E_list.append(e2_manual_ON_0p5)
B = getBmatrix(E_list)
outMatrix(B)

--------------------------------------------------------------------------------
     1               2               3
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00 -1.32563109e-02 -7.91668065e-04
  3   -1.00000000e+00 -7.91668065e-04 -4.44940852e-04

--------------------------------------------------------------------------------


In [37]:
print('Determinant of B', np.linalg.det(B))

Determinant of B 0.012117915660826043


In [38]:
Fe3_headline = 'Fe3 from l501 after NEO contribution:          0:'
Fe3 = ReadLowerTri(fname, Fe3_headline)
Pe3_headline = 'New Electronic  Density3          0:'
Pe3 = ReadLowerTri(fname, Pe3_headline)
e3_headline = 'Electronic Alpha error matrix at iteration for IMat=    3'
e3 = ReadLowerTri(fname, e3_headline)
e3_manual_ON_0p5 = getErrorMatrix(Fe3,Pe3,elec_S,elec_V,0)

In [39]:
E_list.append(e3_manual_ON_0p5)
B = getBmatrix(E_list)
outMatrix(B)

--------------------------------------------------------------------------------
     1               2               3               4
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00 -1.32563109e-02 -7.91668065e-04 -2.33424984e-04
  3   -1.00000000e+00 -7.91668065e-04 -4.44940852e-04  1.02415494e-04
  4   -1.00000000e+00 -2.33424984e-04  1.02415494e-04 -7.35851213e-05

--------------------------------------------------------------------------------


In [40]:
print('Determinant of B', np.linalg.det(B))

Determinant of B -8.765460136836135e-06


In [41]:
Fe4_headline = 'Fe4 from l501 after NEO contribution:          0:'
Fe4 = ReadLowerTri(fname, Fe4_headline)
Pe4_headline = 'New Electronic  Density4          0:'
Pe4 = ReadLowerTri(fname, Pe4_headline)
e4_headline = 'Electronic Alpha error matrix at iteration for IMat=    4'
e4 = ReadLowerTri(fname, e4_headline)
e4_manual_ON_0p5 = getErrorMatrix(Fe4,Pe4,elec_S,elec_V,2)

F
--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -1.10165000e+01 -2.97198000e+00  0.00000000e+00  0.00000000e+00 -7.19961000e-03
  2   -2.97198000e+00 -1.62297000e+00  0.00000000e+00  0.00000000e+00 -1.46741000e-01
  3    0.00000000e+00  0.00000000e+00 -3.60667000e-01  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00 -8.12932000e-02  0.00000000e+00
  5   -7.19961000e-03 -1.46741000e-01  0.00000000e+00  0.00000000e+00 -5.65881000e-01
  6   -3.28294000e-04 -8.03532000e-01  0.00000000e+00  0.00000000e+00 -1.34551000e+00
  7   -4.37578000e-01 -8.91981000e-01  0.00000000e+00  0.00000000e+00 -1.00067000e+00
  8    0.00000000e+00  0.00000000e+00 -2.60843000e-01  0.00000000e+00  0.00000000e+00
  9    0.00000000e+00  0.00000000e+00  0.00000000e+00 -3.67816000e-01  0.00000000e+00
 10    7.29162000e-01  6.56607000e-01  0.00000000e+00  0.00000000e+00  4

In [42]:
E_list.append(e4_manual_ON_0p5)
B = getBmatrix(E_list)
outMatrix(B)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00 -1.32563109e-02 -7.91668065e-04 -2.33424984e-04  2.90719044e-06
  3   -1.00000000e+00 -7.91668065e-04 -4.44940852e-04  1.02415494e-04 -4.94106514e-05
  4   -1.00000000e+00 -2.33424984e-04  1.02415494e-04 -7.35851213e-05  8.62301749e-06
  5   -1.00000000e+00  2.90719044e-06 -4.94106514e-05  8.62301749e-06 -7.87177985e-06

--------------------------------------------------------------------------------


In [43]:
print('Determinant of B', np.linalg.det(B))

Determinant of B 9.048972462343072e-11


In [45]:
Fe5_headline = 'Fe5 from l501 after NEO contribution:          0:'
Fe5 = ReadLowerTri(fname, Fe5_headline)
Pe5_headline = 'New Electronic  Density5          0:'
Pe5 = ReadLowerTri(fname, Pe5_headline)
e5_headline = 'Electronic Alpha error matrix at iteration for IMat=    5'
e5 = ReadLowerTri(fname, e5_headline)
e5_manual_ON_0p5 = getErrorMatrix(Fe5,Pe5,elec_S,elec_V,0)

In [46]:
E_list.append(e5_manual_ON_0p5)
B = getBmatrix(E_list)
outMatrix(B)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00 -1.32563109e-02 -7.91668065e-04 -2.33424984e-04  2.90719044e-06
  3   -1.00000000e+00 -7.91668065e-04 -4.44940852e-04  1.02415494e-04 -4.94106514e-05
  4   -1.00000000e+00 -2.33424984e-04  1.02415494e-04 -7.35851213e-05  8.62301749e-06
  5   -1.00000000e+00  2.90719044e-06 -4.94106514e-05  8.62301749e-06 -7.87177985e-06
  6   -1.00000000e+00  8.92992157e-06 -8.56353269e-06  2.08008764e-06 -1.68235881e-06

     6
  1   -1.00000000e+00
  2    8.92992157e-06
  3   -8.56353269e-06
  4    2.08008764e-06
  5   -1.68235881e-06
  6   -5.05414064e-07

--------------------------------------------------------------------------------


In [47]:
print('Determinant of B', np.linalg.det(B))

Determinant of B -4.0012423268244793e-17


In [55]:
b5_headline = 'B matrix at iteration          5'
b5 = ReadMatrix(fname, b5_headline)

In [56]:
outMatrix(b5)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00 -1.32563000e-02 -7.91890000e-04 -2.33270000e-04  3.24083000e-06
  3   -1.00000000e+00 -7.91890000e-04 -4.44953000e-04  1.02430000e-04 -4.94088000e-05
  4   -1.00000000e+00 -2.33270000e-04  1.02430000e-04 -7.35881000e-05  8.64342000e-06
  5   -1.00000000e+00  3.24083000e-06 -4.94088000e-05  8.64342000e-06 -7.88090000e-06
  6   -1.00000000e+00  8.20038000e-06 -8.58078000e-06  2.05757000e-06 -1.67770000e-06

     6
  1   -1.00000000e+00
  2    8.20038000e-06
  3   -8.58078000e-06
  4    2.05757000e-06
  5   -1.67770000e-06
  6   -5.00992000e-07

--------------------------------------------------------------------------------


In [57]:
print('Determinant of B', np.linalg.det(b5))

Determinant of B -4.1456946341172397e-17


In [58]:
sigma_5b, L_5b  = LA.eig(b5)

In [59]:
sigma_5b

array([ 2.23450509e+00, -2.23764590e+00, -1.02470259e-02, -3.81502210e-04,
       -1.37300274e-05, -1.54475037e-07])

In [60]:
b6_headline = 'B matrix at iteration          6'
b6 = ReadMatrix(fname, b6_headline)

In [61]:
outMatrix(b6)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00 -1.32563000e-02 -7.91890000e-04 -2.33270000e-04  3.24083000e-06
  3   -1.00000000e+00 -7.91890000e-04 -4.44953000e-04  1.02430000e-04 -4.94088000e-05
  4   -1.00000000e+00 -2.33270000e-04  1.02430000e-04 -7.35881000e-05  8.64342000e-06
  5   -1.00000000e+00  3.24083000e-06 -4.94088000e-05  8.64342000e-06 -7.88090000e-06
  6   -1.00000000e+00  8.20038000e-06 -8.58078000e-06  2.05757000e-06 -1.67770000e-06
  7   -1.00000000e+00  7.25023000e-07 -1.49621000e-07  1.70408000e-08 -7.70493000e-08

     6               7
  1   -1.00000000e+00 -1.00000000e+00
  2    8.20038000e-06  7.25023000e-07
  3   -8.58078000e-06 -1.49621000e-07
  4    2.05757000e-06  1.70408000e-08
  5   -1.67770000e-06 -7.70493000e-08
  6   -5.00992000e-07 -3.7726600

In [62]:
print('Determinant of B', np.linalg.det(b6))

Determinant of B 1.8931462781450465e-24


In [63]:
sigma_6b, L_6b  = LA.eig(b6)

In [64]:
sigma_6b

array([ 2.44818708e+00, -2.45080426e+00, -1.07544239e-02, -3.92965714e-04,
       -1.82198624e-05, -4.29195226e-07, -9.54748880e-09])